In [91]:
import bqplot
import ytree
import yt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets
from ipywidgets import *
from bqplot import DateScale, LinearScale, Axis, Lines, Scatter, Bars, Hist, Figure
from bqplot.interacts import (
    FastIntervalSelector, IndexSelector, BrushIntervalSelector,
    BrushSelector, MultiSelector, LassoSelector, PanZoom, HandDraw)


In [92]:
a = ytree.load("rockstar_halos/out_0.list")
# fn = a.save_arbor()
# a = ytree.load(fn)

In [93]:
print(a.derived_field_list)
print(a.field_list)

['halo_id', 'desc_id', 'mass', 'virial_mass', 'virial_radius', 'scale_radius', 'velocity_dispersion', 'position_x', 'position_y', 'position_z', 'velocity_x', 'velocity_y', 'velocity_z', 'angular_momentum_x', 'angular_momentum_y', 'angular_momentum_z', 'spin_parameter', 'T_|U|', 'redshift']
['ID', 'DescID', 'Mvir', 'Vmax', 'Vrms', 'Rvir', 'Rs', 'Np', 'X', 'Y', 'Z', 'VX', 'VY', 'VZ', 'JX', 'JY', 'JZ', 'Spin', 'rs_klypin', 'Mvir_all', 'M200b', 'M200c', 'M500c', 'M2500c', 'Xoff', 'Voff', 'spin_bullock', 'b_to_a', 'c_to_a', 'A[x]', 'A[y]', 'A[z]', 'b_to_a(500c)', 'c_to_a(500c)', 'A[x](500c)', 'A[y](500c)', 'A[z](500c)', 'T/|U|', 'M_pe_Behroozi', 'M_pe_Diemer', 'scale_factor', 'uid', 'desc_uid']


In [94]:
df_trees = {}
counter = 0
for tree in a:
    scale = tree["prog", "scale_factor"]
    assert(scale.size == np.unique(scale).size)
    Mvir = tree["prog", "Mvir_all"]
    Vrms = tree["prog", "Vrms"]
    Rvir = tree["prog", "Rvir"]
    Spin = tree["prog", "Spin"]
    Z = tree["prog", "redshift"]
    ti = str(counter)
    d = pd.DataFrame({('Mvir'): Mvir, ('Vrms'): Vrms, ('Rvir'): Rvir,('Spin'): Spin,('redshift'): Z}, index = scale)
    df_trees[ti] = d
    counter += 1

df = pd.concat(df_trees)

In [88]:
df2 = df.unstack(0)

In [89]:
ind = np.argsort(-df2["Vrms"].loc[1.0])

mvir = df2["Mvir"].iloc[-1]
vrms = df2["Vrms"].iloc[-1]
spin = df2["Spin"].iloc[-1]
rvir = df2["Rvir"].iloc[-1]
z = df2["redshift"].iloc[-1]
mvir = mvir.reindex(df2["Mvir"].columns[ind], axis=1)
vrms = vrms.reindex(df2["Vrms"].columns[ind], axis=1)
spin = spin.reindex(df2["Spin"].columns[ind], axis=1)
rvir = vrms.reindex(df2["Rvir"].columns[ind], axis=1)
z = z.reindex(df2["redshift"].columns[ind], axis=1)

y_axis_dict = {'Rvir':rvir, 'Mvir':mvir,'Spin':spin,'Vrms':vrms,'Redshift':z}

In [105]:
x_scale = bqplot.LinearScale()
y_scale = bqplot.LogScale()

sc_x = LinearScale()
sc_y = LinearScale()
sc_xax = Axis(label=('Vrms'), scale=sc_x, orientation='horizontal')
sc_yax = Axis(label=('Mvir'), scale=sc_y, orientation='vertical')
sc_x2 = LinearScale()
sc_y2 = LinearScale()
sc_xax2 = Axis(label=('Vrms'), scale=sc_x2, orientation='horizontal')
sc_yax2 = Axis(label=('Mvir'), scale=sc_y2, orientation='vertical')


scatter = bqplot.Scatter(x = vrms, y = mvir, 
                        scales = {'x': sc_x, 'y': sc_y}, colors = ['blue'])
scatter2 = bqplot.Scatter(x = vrms, y = mvir,
                        scales = {'x': sc_x2, 'y': sc_y2}, colors = ['red'], visible = False)

db_scat_brush = HTML(value='[]')

chosen_y_axis = mvir
x_y_cond = np.ones(len(mvir),dtype=bool)

# Y-axis selector widget
def dropdown_callback(change):
    chosen_y_axis = y_axis_dict[change['new']]
    scatter2.x = vrms[x_y_cond]
    scatter2.y = chosen_y_axis[x_y_cond]
    fig_scat_brush2.axes =[Axis(label=('Vrms'), scale=sc_x2, orientation='horizontal'),
                           Axis(label=(change['new']), scale=sc_y2, orientation='vertical')]
    fig_scat_brush2.visible = True
    fig_scat_brush2.title = 'Vrms vs. ' + change['new']
    return change['new']

y_options = ['Rvir', 'Mvir','Spin']

y_selector = widgets.Dropdown(
    handler=dropdown_callback,
    options=y_options,
    value='Mvir',
    description='Y Axis:',
    )

y_selector.observe(dropdown_callback, names=['value'])

# Graph brush selector widget
def brush_callback(change):    
    highlight = bselect.selected
    db_scat_brush.value = str(highlight)
    key = y_selector.value
    chosen_y_axis = y_axis_dict[key]
    if change.new is not None:
        try:
            x_condition = (vrms>=float(highlight[0][0])) & (vrms<=float(highlight[1][0]))
            y_condition = (mvir>=float(highlight[0][1])) & (mvir<=float(highlight[1][1]))
            x_y_cond = x_condition & y_condition
            scatter2.x = vrms[x_y_cond]
            scatter2.y = chosen_y_axis[x_y_cond]
            fig_scat_brush2.axes =[Axis(label=('Vrms'), scale=sc_x2, orientation='horizontal'),
                       Axis(label=(key), scale=sc_y2, orientation='vertical')]
            scatter2.visible = True
        except IndexError:
            ;


# Redshift slider widget
redshift_slider = widgets.FloatSlider(value=0.0,min=0.0,max=3.0,step=.01,description='Redshift',
                                      disabled=False,continuous_update=False,
                                      orientation='horizontal',
                                      readout=True,readout_format='.2f',
                                      layout=Layout(width='100%',position='top'))

def on_value_change(change):
    print(change['new'])

redshift_slider.observe(on_value_change, names='value')

bselect = bqplot.interacts.BrushSelector(x_scale=sc_x, y_scale=sc_y, marks=[scatter])

bselect.observe(brush_callback, names=['selected'])

fig_scat_brush1 = Figure(marks=[scatter], axes=[sc_xax, sc_yax], title='Vrms vs. Mvir',
                        interaction=bselect)
fig_scat_brush2 = Figure(marks=[scatter2], axes=[sc_xax2, sc_yax2], title='Vrms vs. Mvir')

left_box = VBox([db_scat_brush, fig_scat_brush1])
right_box = VBox([fig_scat_brush2, y_selector])
graph_box = HBox([left_box, right_box])

VBox([graph_box,redshift_slider])

VBox(children=(HBox(children=(VBox(children=(HTML(value='[]'), Figure(axes=[Axis(label='Vrms', scale=LinearScale()), Axis(label='Mvir', orientation='vertical', scale=LinearScale())], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, interaction=BrushSelector(marks=[Scatter(colors=['blue'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9}, x=array([ 1282.77001953,  1162.29003906,  1156.84997559,  1145.06005859,
        1046.29003906,  1041.26000977,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
         764.47998047,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
         201.38000488,            nan,            nan,            nan,
                  nan,   293.55999756,            nan,   646.58001709,
         641.17999268,            nan,            nan,            nan,
                  nan,   617.11999512,   604.88000488,   603.98999023,
         603.94000244,            nan,            nan,   590.45001221,
         578.02001953,   570.78997803,   568.73999023,            nan,
         564.63000488,   564.36999512,   564.07000732,   558.28997803,
         550.53997803,   549.66998291,   549.41998291,   539.65002441,
         535.7800293 ,   529.65002441,   521.42999268,            nan,
                  nan,            nan,            nan,   509.92001343,
         508.08999634,   503.76998901,   503.13000488,   500.73001099,
                  nan,   493.05999756,   492.82998657,   492.38000488,
         491.10998535,            nan,            nan,   486.33999634,
                  nan,   485.45001221,   484.47000122,   478.77999878,
         977.70001221,            nan,            nan,   468.61999512,
                  nan,   433.92999268,   465.6000061 ,   465.57998657,
         464.30999756,            nan,   457.76000977,   455.01998901,
         454.45001221,   450.61999512,   448.64001465,            nan,
         445.95001221,            nan,   443.08999634,   442.51998901,
         441.76998901,   441.38000488,   439.67001343,   438.29998779,
         437.47000122,   435.97000122,            nan,   432.30999756,
         431.79998779,   430.69000244,   430.29000854,   429.67999268,
         426.8999939 ,   423.5       ,   422.48999023,   422.19000244,
         421.10998535,   418.89001465,            nan,   412.97000122,
         411.98999023,   410.73999023,            nan,   404.79998779,
         403.44000244,   403.07998657,   402.42001343,            nan,
         399.76000977,   390.5       ,   389.1000061 ,   388.29000854,
         385.63000488,   385.60998535,   378.5       ,   376.92999268,
         376.3500061 ,   370.67999268,   367.52999878,   366.57998657,
         365.3500061 ,   365.17999268,   362.70001221,            nan,
         361.70001221,   359.8500061 ,   358.63000488,   357.16000366,
         353.1499939 ,   350.45999146,   348.54000854,   348.11999512,
         347.67999268,   347.52999878,   346.60998535,            nan,
         345.1499939 ,   344.1000061 ,   343.55999756,            nan,
         885.86999512,   335.45999146,   335.36999512,   334.07998657,
         333.89001465,   332.07000732,   331.92001343,   331.60998535,
         330.75      ,   329.73001099,   329.14001465,   327.73001099,
         326.85998535,   325.20001221,   323.48001099,   322.20999146,
         321.48999023,   321.22000122,   319.39001465,   316.98999023,
                  nan,   314.98999023,   314.5       ,   311.6499939 ,
         310.9500